In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from pymongo import MongoClient
import numpy as np
import pandas as pd
import string
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/bcarter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
client = MongoClient()
db =  client.beer_recipes
coll = db.beer_recipes_abv
print(f"The database has {coll.estimated_document_count()} documents.")

The database has 436 documents.


In [24]:
styles_list=db["beer_recipes_abv"].distinct("style")
styles_list

['Cream Ale',
 'Belgian and French Ale',
 'Scottish and Irish Ale',
 'European Amber Lager',
 'Classic American Pilsner',
 'India Pale Ale',
 'Specialty Beer',
 'Schwarzbier',
 'German Wheat and Rye Beer',
 'Amber Hybrid Beer',
 'Stout',
 'Weizenbock',
 'Baltic Porter',
 'Oatmeal Stout',
 'American IPA',
 'American Brown Ale',
 'American Pale Ale',
 'Blonde Ale',
 'Light Lager',
 'American Barleywine',
 'English Brown Ale',
 'Robust Porter',
 'Brown Porter',
 'Saison',
 'Scottish Export 80/-',
 'English Pale Ale',
 'Belgian Specialty Ale',
 'Oktoberfest/Märzen',
 'Belgian Dark Strong Ale',
 'Other Smoked Beer',
 'American Ale',
 'Sweet Stout',
 'Pilsner',
 'Spice/Herb/Vegetable Beer',
 'Special/Best/Premium Bitter',
 'Irish Red Ale',
 'Witbier',
 'English Porter',
 'Weizen/Weissbier',
 'Russian Imperial Stout',
 'Fruit Beer',
 'Doppelbock',
 'Munich Dunkel',
 'American Amber Ale',
 'Belgian Tripel',
 'Gueuze',
 'English Barleywine',
 'Dark Lager',
 'Wood-Aged Beer',
 'Extra Special/Str

In [25]:
df_eda = pd.DataFrame(list(coll.find()))
df_eda['style'].value_counts()

American IPA                              36
Specialty Beer                            25
Saison                                    17
Spice/Herb/Vegetable Beer                 17
American Pale Ale                         15
Fruit Beer                                12
English Brown Ale                         10
Belgian Specialty Ale                      9
Weizen/Weissbier                           9
Oktoberfest/Märzen                         8
Blonde Ale                                 8
American Barleywine                        8
India Pale Ale                             7
Baltic Porter                              7
Russian Imperial Stout                     7
Belgian and French Ale                     7
Amber Hybrid Beer                          6
Extra Special/Strong Bitter                6
English Barleywine                         6
Brown Porter                               6
Light Lager                                6
Imperial IPA                               6
American A

In [26]:
recipe_list=[]
for x in coll.find({}, {"recipe": 1 , "keywords": 1, "_id": 0}):
   recipe_list.append((x['recipe'], str(x['keywords']).strip('[]')))
labels = ['recipe', 'keywords']
df = pd.DataFrame.from_records(recipe_list, columns=labels)
df.info(),

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 2 columns):
recipe      436 non-null object
keywords    436 non-null object
dtypes: object(2)
memory usage: 6.9+ KB


(None,)

In [27]:
df_eda[["recipe","style"]]

,recipe,style
0,$12 Cream Ale,Cream Ale
1,Lampo Bianco (White Lightning),Belgian and French Ale
2,McZainasheff’s Wee,Scottish and Irish Ale
3,Munich Madness,European Amber Lager
4,Not-So Pilsener,Classic American Pilsner
5,Pliny the Toddler,India Pale Ale
6,Poor Richard’s Ale,Specialty Beer
7,Puna Coast Black Lager,Schwarzbier
8,Roggen (2010),German Wheat and Rye Beer
9,Sticke Alt,Amber Hybrid Beer


In [28]:
df.keywords

0                     '5.12', '20.7', '3.6', 'Cream Ale'
1           '5.8', '19', '5.6', 'Belgian and French Ale'
2            '9.7', '28', '16', 'Scottish and Irish Ale'
3             '4.6', '29', '9.8', 'European Amber Lager'
4      '4.7', '40', '5(10EBC)', 'Classic American Pil...
5                   '4.7', '56', '3.7', 'India Pale Ale'
6                '6.5', '26.9', '18.8', 'Specialty Beer'
7                '4.8', '22', '28(56EBC)', 'Schwarzbier'
8      '5.55', '19.3', '14.2', 'German Wheat and Rye ...
9              '5.3*', '50*', '22*', 'Amber Hybrid Beer'
10     '8.0', '23', '16(31EBC)', 'German Wheat and Ry...
11                     '4.9', '29', '39(78EBC)', 'Stout'
12                       '8.0', '23', '16', 'Weizenbock'
13     '5-5.6', '30', 'Paleamber', 'Belgian and Frenc...
14                    '9.4', '38', '28', 'Baltic Porter'
15                 '7.20', '30.5', '49', 'Oatmeal Stout'
16                     '6.7', '55', '10', 'American IPA'
17               '5.5', '28', '

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df.keywords)
tfidf_matrix.shape

(436, 212)

In [31]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(436, 436)

In [40]:
indices = pd.Series(df.index, index=df['recipe']).drop_duplicates()
for row in sorted(indices.iteritems()):
    print(row)

('#BiggestSmallBeerEver Porter', 27)
('$12 Cream Ale', 0)
('1834 Augsburger Lagerbier', 160)
('1879 Kulmbacher Export', 209)
('2010 Barleywine', 302)
('2015 Big Brew Recipe #2: Old School Barleywine', 23)
('2015 Big Brew Recipe #3: Killer Kölsch', 396)
('7k Czech Pale Lager', 22)
('80 Shilling Scottish Ale', 29)
('A Festbier For the Rest Of Us', 35)
('AG24 Oktoberfest', 59)
('Abbey Saison', 44)
('Adroit Theory Triptych Tart Rambutan Saison', 54)
('Alabama Hopslammer American Pale Ale', 62)
('Albanach Láidir', 75)
('Ale of Asgard', 84)
('Allagash White Clone', 89)
('Amahl’s English Brown Ale', 102)
('Amber Waves of Grain', 115)
('American IPA con Pomelo Rosado (Pink Grapefruit American IPA)', 129)
('American Malt Liquor (Olde English Clone)', 132)
('American-Style Cascadian India Dark Double Black IPA (ASCIDDBIPA)', 139)
('Anchor Brewing Old Foghorn', 153)
('Anchor Steam Clone', 161)
('Ardley Brown Ale', 187)
('Ass Kicking Cream Ale (Tequila-aged Lime/Jalapeno Ale)', 199)
('Atlas Brew W

In [33]:
def get_recommendations(recipe, cosine_sim=cosine_sim):
    
    idx = indices[recipe]
    sim_scores = list(enumerate(cosine_sim[idx]))
    if len(sim_scores) >= 5:
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]
    else:
        return('No Matches')
    beer_indices = [i[0] for i in sim_scores]
    beer_scores = [i[1] for i in sim_scores]
    return df['recipe'].iloc[beer_indices], beer_scores
    

In [34]:
get_recommendations('Bell’s Two Hearted Ale Clone')

(156           Great South Bay Brewery Massive IPA
 268                      Nuclear Winter White IPA
 356                           Smoked Habanero IPA
 290    Peaks &amp; Priorities Juicy IPA (Extract)
 262                           No Fail Extract IPA
 Name: recipe, dtype: object,
 [0.9999999999999999,
  0.708678180565044,
  0.6521247230513623,
  0.6394910092292164,
  0.6197265763509396])

In [35]:
get_recommendations('Bill’s Farmhouse Ale')

(141              Funkwerks Saison
 234              Mad About Saison
 239    Melissa and Patty’s Saison
 177            Honey Squished IPA
 44                   Abbey Saison
 Name: recipe, dtype: object,
 [0.4560680324002677,
  0.45192716676524486,
  0.45192716676524486,
  0.4404760266916438,
  0.4314591310144279])

In [36]:
get_recommendations('Big Brew 2013 Recipe: Pointon Proper')

(102            Amahl’s English Brown Ale
 258    Nick’s Northern English Brown Ale
 68                 Choconut Brown Porter
 187                     Ardley Brown Ale
 78            Coriandered Mild Brown Ale
 Name: recipe, dtype: object,
 [0.778791022246199,
  0.6140839914517675,
  0.5737547581241026,
  0.5555795721124648,
  0.5198978518604497])

In [39]:
get_recommendations('Allagash White Clone')

(424                              Hoegaarden White Clone
 55                            Brunchmaster 2000 Witbier
 291    Pedal Haus Brewery Biere Blanche Belgian Witbier
 158                                  Grisette de Saison
 280                                   Ordinary Grisette
 Name: recipe, dtype: object,
 [0.9999999999999999,
  0.6580896489372537,
  0.5813864341793236,
  0.30609843062767883,
  0.2917385326344214])

In [38]:
print (sorted(tfidf.vocabulary_))

['01', '03', '04', '06', '10', '100', '103', '107', '10ebc', '11', '119', '12', '120', '12ebc', '13', '14', '15', '16', '164', '16ebc', '17', '18', '19', '20', '200', '20ebc', '21', '22', '23', '24', '25', '26', '27', '28', '284calculated', '29', '30', '31', '31ebc', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '55butperceptionmaybegreater', '56', '56ebc', '57', '58', '59', '5ebc', '60', '63', '64', '65', '66', '67', '68', '69', '70', '72', '74', '75', '76', '78', '78ebc', '80', '81', '82', '83', '85', '86', '87', '89', '90', '93', '94', '95', '96', '99', 'actual', 'aged', 'ale', 'altbier', 'amber', 'american', 'baltic', 'barleywine', 'beer', 'belgian', 'berliner', 'best', 'bitter', 'bière', 'blackasamoonlessnight', 'blond', 'blonde', 'bock', 'bohemian', 'brown', 'bruin', 'butperceptionmaybegreater', 'calculated', 'california', 'christmas', 'classic', 'common', 'cream', 'dark', 'deepamber